Creating the backprop

https://github.com/lazyprogrammer/machine_learning_examples/blob/master/ann_class/backprop.py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def forward(X, W1, b1, W2, b2):
    Z = 1 / (1 + np.exp(-X.dot(W1) - b1))
    A = Z.dot(W2) + b2
    expA = np.exp(A)
    Y = expA / expA.sum(axis=1, keepdims=True)
    return Y, Z

def classification_rate(Y, P):
    n_correct = 0
    n_total = 0
    for i in xrange(len(Y)):
        n_total += 1
        if Y[i] == P[i]:
            n_correct += 1
    return float(n_correct) / n_total

In [ ]:
def main():
     # create the data
    Nclass = 500
    D = 2 # dimensionality of input (# of features)
    M = 3 # hidden layer size
    K = 3 # number of classes (outputs)
    
    X1 = np.random.randn(Nclass, D) + np.array([0, -2])
    X2 = np.random.randn(Nclass, D) + np.array([2, 2])
    X3 = np.random.randn(Nclass, D) + np.array([-2, 2])
    X = np.vstack([X1, X2, X3]) 
    
    Y = np.array([0]*Nclass + [1]*Nclass + [2]*Nclass) # A 1500X1 array - the classification(targets)
    N = len(Y) # 1500
    
    # turn Y into an indicator matrix for training
    T = np.zeros((N,K)) #Indicator variable
    
    # The for loop uses Y, a 1500X1 array and creates "T" a one-hot encoded 1500X3 matrix for the targets
    for i in xrange(N):
        T[i, Y[i]] = 1
        
    #Visualize Data
    plt.scatter(X[:,0], X[:,1], c=Y, s=100, alpha=0.5)
    plt.show()
    
    #Randomly initialize weights
    W1 = np.random.rand(D,M)
    b1 = np.random.rand(M)
    W2 = np.random.rand(M,K)
    b2 = np.random.rand(K) 
    
    #Now for back prop
    
    learning_rate = 10e-7
    costs = [] #Keep an array of the cost function so we can see the progression afterwards
    
    for epoch in xrange(100000): #how many times we are going to cycle
        ouput, hidden = forward(X, W1, b1, W2, b2) #hidden = Z. Required to calculate the gradient
        if epoch % 100 == 0: #every 100 epochs we are going to calculate the cost 
            c = cost(T, ouput) #(not yet written)
            P = np.argmax(output, axis = 1)
            
    
    #3:19

In [14]:
Nclass = 500
D = 2 # dimensionality of input (# of features)
M = 3 # hidden layer size
K = 3 # number of classes (outputs)
    
X1 = np.random.randn(Nclass, D) + np.array([0, -2])
X2 = np.random.randn(Nclass, D) + np.array([2, 2])
X3 = np.random.randn(Nclass, D) + np.array([-2, 2])
X = np.vstack([X1, X2, X3]) # a 1500x2 matrix
    
Y = np.array([0]*Nclass + [1]*Nclass + [2]*Nclass) # A 1500X1 array
N = len(Y) # 1500
T = np.zeros((N,K))

In [16]:
T.shape

(1500L, 3L)

In [17]:
for i in xrange(N):
        T[i, Y[i]] = 1


In [21]:
T

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [3]:
# __main__ — Top-level script environment

# This module represents the (otherwise anonymous) scope in which the interpreter’s main program executes — commands read 
# either from standard input, from a script file, or from an interactive prompt. It is this environment in which the idiomatic 
# “conditional script” stanza causes a script to run:

if __name__ == '__main__':
    main()

NameError: name 'main' is not defined